In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline

In [54]:
data = './dataset/finalRoad.csv'
df = pd.read_csv(data)
df.head

<bound method NDFrame.head of        Accident_Index  longitude   latitude  Accident_Severity  Day_of_Week  \
0                   1  78.610393  14.724026                  2            3   
1                   2  78.534042  14.762353                  3            4   
2                   3  78.470877  14.745606                  3            5   
3                   4  78.557994  14.667128                  3            6   
4                   5  78.576431  14.703443                  3            2   
...               ...        ...        ...                ...          ...   
59993           59994  70.144710  24.586263                  3            4   
59994           59995  87.530231  25.803174                  3            4   
59995           59996  70.546520  25.663263                  3            4   
59996           59997  77.030311  25.308850                  3            4   
59997           59998  69.094869  21.735663                  3            4   

        Time  Speed_l

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59998 entries, 0 to 59997
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Accident_Index      59998 non-null  int64  
 1   longitude           59998 non-null  float64
 2   latitude            59998 non-null  float64
 3   Accident_Severity   59998 non-null  int64  
 4   Day_of_Week         59998 non-null  int64  
 5   Time                59997 non-null  object 
 6   Speed_limit         59998 non-null  int64  
 7   Light_Conditions    59998 non-null  int64  
 8   Weather_Conditions  59998 non-null  int64  
 9   Date                59998 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 4.6+ MB


In [56]:
df.isnull().sum()

Accident_Index        0
longitude             0
latitude              0
Accident_Severity     0
Day_of_Week           0
Time                  1
Speed_limit           0
Light_Conditions      0
Weather_Conditions    0
Date                  0
dtype: int64

In [57]:
X = df.drop(['Accident_Index','Accident_Severity', "Time", "Date"], axis = 1)
y = df['Accident_Severity']

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [59]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
test_acc_knn = round(knn.score(X_test, y_test) * 100, 2)
# acc_knn
print('Train Score: ', acc_knn)  
print('Test Score: ', test_acc_knn) 

Train Score:  88.53
Test Score:  82.25


In [60]:
import urllib.request
import json
import time

In [61]:
channel_ID = "2061500"

mqtt_host = "mqtt3.thingspeak.com"

mqtt_client_ID = "PCEfJSM9Cwo3BSYULiwnNCw"
mqtt_username  = "PCEfJSM9Cwo3BSYULiwnNCw"
mqtt_password  = "OlAVymnEWY4mTMtfc5X5p37F"

t_transport = "websockets"
t_port = 80

topic = "channels/" + "2061500" + "/subscribe"

In [62]:
READ_API_KEY='CG50D1N7SZQ4ETMZ'
CHANNEL_ID= '2061500'
# global TS

In [74]:
# for i in range(10):
TS = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" %(CHANNEL_ID,READ_API_KEY))
response = TS.read()
data=json.loads(response)


In [64]:
from datetime import datetime
a = data['created_at'][:10]
# print (a)
date_obj = datetime.strptime(a, '%Y-%m-%d').date()
day = date_obj.weekday()
day

5

In [69]:
X_test

,longitude,latitude,Day_of_Week,Speed_limit,Light_Conditions,Weather_Conditions
10327,87.470136,25.792640,6,30,1,1
11900,82.151626,22.088755,6,30,4,1
41557,77.198831,28.541666,7,30,1,1
49312,73.802769,18.634977,5,30,1,1
28440,75.888848,22.722887,3,30,1,1
...,...,...,...,...,...,...
1237,78.608020,14.772135,2,30,1,1
48352,73.903477,18.577422,3,70,1,2
7208,94.217934,26.741343,6,30,4,2
59684,83.819224,20.929772,6,60,1,2


In [ ]:
accident = []
accident.append(data["feild3"])
accident.append(data["feild4"])
accident.append(day)
light = data["feild2"]
if light >1000:
    accident.append(7)
elif light < 50:
    accident.append(1)
else:
    accident.append(4)

if data["field2"] >500 and data["field2"]<1000:
    accident.append(7)
elif data['field6'] == 1 and data["field5"]<=0:
    accident.append(8)
elif data['feil6'] == 0 and data["field5"]<=0:
    accident.append(3)
elif data['field6'] == 1 and data["field5"]>0 and data["field5"]<20:
    accident.append(6)
elif data["field5"]>10 and data["field5"]<35:
    accident.append(5)
elif data['field5'] == 1:
    accident.append(9)
else:
    accident.append(1)

In [75]:
data

{'created_at': '2023-03-12T07:54:51Z',
 'entry_id': 307,
 'field1': None,
 'field2': '73',
 'field3': None,
 'field4': None,
 'field5': None,
 'field6': None}

In [66]:
TS.close()

In [67]:
X_test

,longitude,latitude,Day_of_Week,Speed_limit,Light_Conditions,Weather_Conditions
10327,87.470136,25.792640,6,30,1,1
11900,82.151626,22.088755,6,30,4,1
41557,77.198831,28.541666,7,30,1,1
49312,73.802769,18.634977,5,30,1,1
28440,75.888848,22.722887,3,30,1,1
...,...,...,...,...,...,...
1237,78.608020,14.772135,2,30,1,1
48352,73.903477,18.577422,3,70,1,2
7208,94.217934,26.741343,6,30,4,2
59684,83.819224,20.929772,6,60,1,2
